In [1]:
# Upload flight delay data and template files
!ls

Airline_Delay_Cause.csv		     query_intent_status_template_3.csv
query_intent_booking_template_3.csv  sample_data
query_intent_general_template_3.csv


In [7]:
# Create a dictionary of unique carrier codes and names
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('Airline_Delay_Cause.csv')

# Create a DF of unique carriers only
unique_airlines = df[['carrier', 'carrier_name']].drop_duplicates()

# Create a dictionary of unique carriers
airline_dict = dict(zip(unique_airlines['carrier'], unique_airlines['carrier_name']))

# Display the unique carriers DF
unique_airlines.head()

,carrier,carrier_name
0,9E,Endeavor Air Inc.
92,AA,American Airlines Inc.
220,AS,Alaska Airlines Inc.
308,B6,JetBlue Airways
376,DL,Delta Air Lines Inc.


In [9]:
# Display the shape of the unique carriers DF
unique_airlines.shape, len(airline_dict)

((23, 2), 21)

In [10]:
# Take out 'Inc.', 'Co.' and 'Corp'
for key, value in airline_dict.items():
    idx = value.find("Inc")
    if idx != -1:
        airline_dict[key] = value[: idx - 1]
    else:
        idx = value.find("Co")
        if idx != -1:
            airline_dict[key] = value[: idx - 1]

# Split airline names
for key, value in airline_dict.items():
    airline_dict[key] = value.split()

# Make 'American Eagle' one string
one_str = airline_dict["MQ"][0] + " " + airline_dict["MQ"][1]
airline_dict["MQ"][0] = one_str
del airline_dict["MQ"][1]

# Make 'Air Lines' one string to match csv data and maybe help training
one_str = airline_dict["DL"][1] + " " + airline_dict["DL"][2]
airline_dict["DL"][1] = one_str
del airline_dict["DL"][2]

one_str = airline_dict["UA"][1] + " " + airline_dict["UA"][2]
airline_dict["UA"][1] = one_str
del airline_dict["UA"][2]

one_str = airline_dict["NK"][1] + " " + airline_dict["NK"][2]
airline_dict["NK"][1] = one_str
del airline_dict["NK"][2]

airline_dict, len(airline_dict)

({'9E': ['Endeavor', 'Air'],
  'AA': ['American', 'Airlines'],
  'AS': ['Alaska', 'Airlines'],
  'B6': ['JetBlue', 'Airways'],
  'DL': ['Delta', 'Air Lines'],
  'F9': ['Frontier', 'Airlines'],
  'G4': ['Allegiant', 'Air'],
  'HA': ['Hawaiian', 'Airlines'],
  'MQ': ['American Eagle', 'Airlines'],
  'NK': ['Spirit', 'Air Lines'],
  'OH': ['PSA', 'Airlines'],
  'OO': ['SkyWest', 'Airlines'],
  'UA': ['United', 'Air Lines'],
  'WN': ['Southwest', 'Airlines'],
  'YX': ['Republic', 'Airline'],
  'QX': ['Horizon', 'Air'],
  'YV': ['Mesa', 'Airlines'],
  'EV': ['ExpressJet', 'Airlines'],
  'VX': ['Virgin', 'America'],
  'US': ['US', 'Airways'],
  'FL': ['AirTran', 'Airways']},
 21)

In [11]:
# Create a list of carrier codes
carrier_codes = list(airline_dict.keys())
carrier_codes, len(carrier_codes)

(['9E',
  'AA',
  'AS',
  'B6',
  'DL',
  'F9',
  'G4',
  'HA',
  'MQ',
  'NK',
  'OH',
  'OO',
  'UA',
  'WN',
  'YX',
  'QX',
  'YV',
  'EV',
  'VX',
  'US',
  'FL'],
 21)

In [12]:
# Create a DF of unique airports only
unique_airports = df[['airport', 'airport_name']].drop_duplicates()

# Create a dictionary of unique airports
airport_dict = dict(zip(unique_airports['airport'], unique_airports['airport_name']))

# Display the unique airports DF
unique_airports

,airport,airport_name
0,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ..."
1,ABY,"Albany, GA: Southwest Georgia Regional"
2,AEX,"Alexandria, LA: Alexandria International"
3,AGS,"Augusta, GA: Augusta Regional at Bush Field"
4,ALB,"Albany, NY: Albany International"
...,...,...
151732,CIC,"Chico, CA: Chico Municipal"
154943,FOE,"Topeka, KS: Topeka Regional"
158861,LMT,"Klamath Falls, OR: Crater Lake Klamath Regional"
158864,MOD,"Modesto, CA: Modesto City-County-Harry Sham Field"


In [13]:
# Display the airport dictionary size, keys and values
len(airport_dict)#, airport_dict

395

In [14]:
airport_name = list(airport_dict.values())
airport_name[0]

'Allentown/Bethlehem/Easton, PA: Lehigh Valley International'

In [15]:
import re
name_list = re.split(r"[/,]", airport_name[0])
name_list

['Allentown', 'Bethlehem', 'Easton', ' PA: Lehigh Valley International']

In [16]:
name_list[0]

'Allentown'

In [17]:
# Data for building random dates
NUM_text = ['one','two','three','four','five','six','seven','eight','nine','ten']
DDD_list = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
WEEKDAY_list = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
MMM_list = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
MONTH_list = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]
DDth_list = [
    "1st",
    "2nd",
    "3rd",
    "4th",
    "5th",
    "6th",
    "7th",
    "8th",
    "9th",
    "10th",
    "11th",
    "12th",
    "13th",
    "14th",
    "15th",
    "16th",
    "17th",
    "18th",
    "19th",
    "20th",
    "21st",
    "22nd",
    "23rd",
    "24th",
    "25th",
    "26th",
    "27th",
    "28th",
]

### Build Altered CSV file

In [44]:
import csv
import random
import string
from datetime import datetime


def build_altered_csv(template_file, intent):
    """
    Add random data to the template csv file variables, shuffle
    the altered file and download it.
    Args:
        template_file (str): The name of the template csv file.
        intent (str): The intent of the csv file.
    """
    airport = list(airport_dict.keys())
    airport_name = list(airport_dict.values())

    # Get current date and time
    current_datetime = datetime.now()
    current_year = current_datetime.year
    current_month = current_datetime.month
    current_day = current_datetime.day

    # Create a list to save altered csv data
    altered_data = []

    # Open the CSV file in read mode ('r')
    with open(template_file, "r", newline="") as csvfile:

        reader = csv.reader(csvfile)  # Create a CSV reader object
        next(reader)  # Skip header
        i = -1  # index for carrier_codes
        cnt = 0  # counter of rows for debugging

        # Iterate through each row in the CSV file
        for row in reader:

            if i < len(carrier_codes) - 1:
                i += 1
            else:
                i = 0

            # Alter the query
            query = row[0].split()

            i1 = random.randint(0, len(airport_dict) - 1)
            i2 = random.randint(0, len(airport_dict) - 1)

            for _ in range(2):
                if "airptCd1" in query:
                    idx = query.index("airptCd1")
                    query[idx] = airport[i1]  # Get first airport code
                elif "airptNm1" in query:
                    idx = query.index("airptNm1")
                    name_list = re.split(r"[/,]", airport_name[i1])
                    query[idx] = name_list[0]  # Get first airport name

                if "airptCd2" in query:
                    idx = query.index("airptCd2")
                    query[idx] = airport[i2]  # Get second airport code
                elif "airptNm2" in query:
                    idx = query.index("airptNm2")
                    name_list = re.split(r"[/,]", airport_name[i2])
                    query[idx] = name_list[0]  # Get second airport name

            if "MM" in query:
                idx = query.index("MM")
                MM_idx = random.randint(1, 12)
                query[idx] = str(MM_idx)  # Get numeric month
            elif "MMM" in query:
                idx = query.index("MMM")
                MM_idx = random.randint(1, 12)
                query[idx] = MMM_list[MM_idx - 1]  # Get abbreviated month
            elif "MONTH" in query:
                idx = query.index("MONTH")
                MM_idx = random.randint(1, 12)
                query[idx] = MONTH_list[MM_idx - 1]  # Get abbreviated month

            if "DD" in query:
                idx = query.index("DD")
                high_DD = 31
                if MM_idx == 2:
                    high_DD = 28
                elif MM_idx == 4 or MM_idx == 6 or MM_idx == 9 or MM_idx == 11:
                    high_DD = 30
                DD_idx = random.randint(1, high_DD)
                query[idx] = str(DD_idx)  # Get numeric date
            elif "DDth" in query:
                idx = query.index("DDth")
                DD_idx = random.randint(1, 28)
                query[idx] = DDth_list[DD_idx - 1]  # Get ordinal date
            elif "DDD" in query:
                idx = query.index("DDD")
                DDD_idx = random.randint(0, 6)
                query[idx] = DDD_list[DDD_idx]  # Get abbreviated day
            elif "WEEKDAY" in query:
                idx = query.index("WEEKDAY")
                DDD_idx = random.randint(0, 6)
                query[idx] = WEEKDAY_list[DDD_idx]  # Get abbreviated day

            if "YY" in query or "YYYY" in query:
                if (
                    current_month == MM_idx and current_day >= DD_idx
                ) or current_month > MM_idx:
                    current_year += 1

                if "YY" in query:
                    idx = query.index("YY")
                    query[idx] = str(current_year)[
                        -2:
                    ]  # Get last two digits of current year
                else:
                    idx = query.index("YYYY")
                    query[idx] = str(current_year)  # Get current year

                current_year = current_datetime.year  # reset for next loop

            if "NUM" in query:
                idx = query.index("NUM")
                NUM_idx = random.randint(0, 9)
                query[idx] = NUM_text[NUM_idx]  # Get number

            if "CC" in query:
                idx = query.index("CC")
                query[idx] = carrier_codes[i]

            if "cc" in query:
                idx = query.index("cc")
                query[idx] = carrier_codes[i].lower()

            if "Capcxr1" in query:
                idx = query.index("Capcxr1")
                query[idx] = airline_dict[carrier_codes[i]][
                    0
                ]  # Get first name of carrier capitalized

            if "Capcxr2" in query:
                idx = query.index("Capcxr2")
                query[idx] = airline_dict[carrier_codes[i]][
                    1
                ]  # Get second name of carrier capitalized

            if "lowcxr1" in query:
                idx = query.index("lowcxr1")
                query[idx] = airline_dict[carrier_codes[i]][
                    0
                ].lower()  # Get first name of carrier lowercase

            if "lowcxr2" in query:
                idx = query.index("lowcxr2")
                query[idx] = airline_dict[carrier_codes[i]][
                    1
                ].lower()  # Get second name of carrier lowercase

            if "9999_" in query:
                idx = query.index("9999_")
                query[idx] = str(random.randint(1, 9999))  # Get a random flight number

            if "9999_?" in query:
                idx = query.index("9999_?")
                query[idx] = (
                    str(random.randint(1, 9999)) + "?"
                )  # Get a random flight number

            if "ABC123" in query:
                idx = query.index("ABC123")
                random_indices = [random.randint(1, 5), random.randint(1, 5)]
                if random_indices[0] == random_indices[1]:
                    if random_indices[0] == 5:
                        random_indices[1] -= 1
                    else:
                        random_indices[1] += 1
                pnr = ""
                for i in range(6):
                    if i == random_indices[0] or i == random_indices[1]:
                        pnr += str(random.randint(1, 9))
                    else:
                        pnr += random.choice(string.ascii_lowercase)
                query[idx] = pnr  # Get a random PNR (Passenger Name Record locator)

            for _ in range(3):
                if "+-" in query:
                    idx = query.index("+-")
                    query[idx - 1] = query[idx - 1] + "-" + query[idx + 1]
                    del query[idx : idx + 2]

                if "+/" in query:
                    idx = query.index("+/")
                    query[idx - 1] = query[idx - 1] + "/" + query[idx + 1]
                    del query[idx : idx + 2]

                if "'s" in query:
                    idx = query.index("'s")
                    query[idx - 1] = query[idx - 1] + "'s"
                    del query[idx]

                if "," in query:
                    idx = query.index(",")
                    query[idx - 1] = query[idx - 1] + ","
                    del query[idx]

                if "." in query:
                    idx = query.index(".")
                    query[idx - 1] = query[idx - 1] + "."
                    del query[idx]

                if "?" in query:
                    idx = query.index("?")
                    query[idx - 1] = query[idx - 1] + "?"
                    del query[idx]

            for _ in range(3):
                if "+" in query:
                    idx = query.index("+")
                    query[idx - 1] = query[idx - 1] + query[idx + 1]
                    del query[idx : idx + 2]

            joined_query = " ".join(query)
            altered_data.append([joined_query, intent])

            cnt += 1

        # End of for loop

        # Shuffle the list a few times
        random.shuffle(altered_data)
        random.shuffle(altered_data)
        random.shuffle(altered_data)

        # Add the header labels
        altered_data.insert(0, ["query", "intent"])
        len(altered_data)

        # Create a new csv file
        altered_file = "query_intent_" + intent + "_altered.csv"

        # Write the altered data to the new csv file
        with open(altered_file, "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(altered_data)

        # Save the file locally to my machine
        from google.colab import files

        files.download(altered_file)

### Build Booking file

In [45]:
template_file = 'query_intent_booking_template_3.csv'
intent = 'booking'
build_altered_csv(template_file, intent)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Build Status file

In [46]:
template_file = 'query_intent_status_template_3.csv'
intent = 'status'
build_altered_csv(template_file, intent)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Build General file

In [47]:
template_file = 'query_intent_general_template_3.csv'
intent = 'general'
build_altered_csv(template_file, intent)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>